## Read the table  from the "silver" database

In [0]:
transaction_df=spark.read.table("hive_metastore.silver.cleaned_transaction");
customer_df=spark.read.table("hive_metastore.silver.cleaned_customer");
branch_df=spark.read.table("hive_metastore.silver.cleaned_branch");
merged_df=spark.read.table("hive_metastore.silver.cleaned_mereged");

## Fraud_Flags_table

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType, FloatType, StructType, StructField

# Assume merged_df is already defined and contains the necessary columns

# Calculate the transaction count for each customer
transaction_counts_df = merged_df.groupBy("customer_id").agg(F.count("transaction_id").alias("transaction_count"))

# Join the transaction count back to the main DataFrame
joined_with_counts_df = merged_df.join(transaction_counts_df, on="customer_id", how="left")

# Define the UDF to classify transactions
def classify_transaction(amount, zipcode, transaction_count):
    results = []
    if amount > 1000:
        results.append(("unusual_amount", 0.90))
    if amount < 5:
        results.append(("low_value_transaction", 0.60))
    if transaction_count > 10:
        results.append(("watchlist_match", 0.95))
    return results

# Register the UDF with the correct return type
classify_transaction_udf = F.udf(classify_transaction, ArrayType(StructType([
    StructField("flag_type", StringType(), True),
    StructField("confidence_score", FloatType(), True)
])))

# Apply the UDF to classify transactions and explode the results
classified_df = joined_with_counts_df.withColumn(
    "flags", F.explode(classify_transaction_udf(F.col("amount"), F.col("Zip_code"), F.col("transaction_count")))
)

# Extract the relevant columns and create a unique flag ID
fraud_flag_df = classified_df.select(
    F.monotonically_increasing_id().alias("flag_id"),
    F.col("transaction_id"),
    F.col("flags.flag_type").alias("flag_type"),
    F.round(F.col("flags.confidence_score"), 2).alias("confidence_score"),
    F.col("timestamp")
)

# Convert the flag_id to a formatted string
fraud_flag_df = fraud_flag_df.withColumn("flag_id", F.concat(F.lit("F00"), F.col("flag_id").cast("string")))

# Display the DataFrame with the new flag details
fraud_flag_df.display(truncate=False)


flag_id,transaction_id,flag_type,confidence_score,timestamp
F000,T6067,low_value_transaction,0.6,2023-01-22T16:15:00Z
F001,T6074,unusual_amount,0.9,2023-01-22T19:12:00Z
F002,T6082,low_value_transaction,0.6,2023-01-22T22:58:00Z
F003,T6086,watchlist_match,0.95,2023-01-23T01:11:00Z
F004,T6094,unusual_amount,0.9,2023-01-23T07:07:00Z
F005,T6099,low_value_transaction,0.6,2023-01-23T09:01:00Z
F006,T6102,unusual_amount,0.9,2023-01-23T10:30:00Z
F007,T6105,unusual_amount,0.9,2023-01-23T11:52:00Z
F008,T6111,low_value_transaction,0.6,2023-01-23T14:35:00Z
F009,T6113,unusual_amount,0.9,2023-01-23T15:37:00Z


In [0]:
fraud_flag_df.write.format("delta").mode("overwrite").saveAsTable("hive_metastore.gold.Fraud_Flag")

## Customer_segments_Table

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from datetime import datetime, timedelta
import pandas as pd

# Current date
current_date = pd.Timestamp("2023-01-01")

# High_Value: Customers with total transaction amount above the threshold
high_value_threshold = 100000  
high_value_customers = merged_df.groupBy("customer_id").agg(F.sum("amount").alias("total_amount"))
high_value_customers = high_value_customers.filter(F.col("total_amount") > high_value_threshold).select(
    "customer_id"
).withColumn("segment_name", F.lit("High_Value")).withColumn(
    "segment_description", F.lit("Customers with high transaction volume")
).withColumn(
    "last_update", F.lit(current_date)
)

# New_User: Customers who joined in the last 30 days
new_user_customers = merged_df.filter(
    F.col("join_date") > F.lit(current_date - timedelta(days=30))
).select("customer_id").withColumn(
    "segment_name", F.lit("New_User")
).withColumn(
    "segment_description", F.lit("Customers who joined in last 30 days")
).withColumn(
    "last_update", F.lit(current_date)
)

# Inactive: Customers with no transactions in last 90 days
recent_transactions = merged_df.filter(
    F.col("timestamp") > F.lit(current_date - timedelta(days=90))
).select("customer_id").distinct()
inactive_customers = merged_df.join(recent_transactions, on="customer_id", how="left_anti").select(
    "customer_id"
).withColumn(
    "segment_name", F.lit("Inactive")
).withColumn(
    "segment_description", F.lit("No transactions in last 90 days")
).withColumn(
    "last_update", F.lit(current_date)
)

# Credit_Risk: Customers with low credit scores (assuming credit_score < 600)
credit_risk_customers = merged_df.filter(F.col("credit_score") < 600).select(
    "customer_id"
).withColumn(
    "segment_name", F.lit("Credit_Risk")
).withColumn(
    "segment_description", F.lit("Customers with low credit scores")
).withColumn(
    "last_update", F.lit(current_date)
)

# Loyal: Customers with consistent activity for over 5 years
loyal_customers = merged_df.filter(
    F.col("join_date") < F.lit(current_date - timedelta(days=5*365))
).select("customer_id").withColumn(
    "segment_name", F.lit("Loyal")
).withColumn(
    "segment_description", F.lit("Consistent activity for over 5 years")
).withColumn(
    "last_update", F.lit(current_date)
)

# Combine all segments
customer_segmentation_df = high_value_customers.union(new_user_customers).union(inactive_customers).union(credit_risk_customers).union(loyal_customers)

# Add segment_id
window_spec = Window.orderBy("customer_id")
customer_segmentation_df = customer_segmentation_df.withColumn("segment_id", F.concat(F.lit("S00"), F.row_number().over(window_spec).cast("string")))

# Select and display the result
customer_segmentation_df = customer_segmentation_df.select("segment_id", "customer_id", "segment_name", "segment_description", "last_update")

customer_segmentation_df.display()


segment_id,customer_id,segment_name,segment_description,last_update
S001,C1000,Credit_Risk,Customers with low credit scores,2023-01-01T00:00:00Z
S002,C1000,Credit_Risk,Customers with low credit scores,2023-01-01T00:00:00Z
S003,C1000,Credit_Risk,Customers with low credit scores,2023-01-01T00:00:00Z
S004,C1000,Loyal,Consistent activity for over 5 years,2023-01-01T00:00:00Z
S005,C1000,Loyal,Consistent activity for over 5 years,2023-01-01T00:00:00Z
S006,C1000,Loyal,Consistent activity for over 5 years,2023-01-01T00:00:00Z
S007,C1001,Credit_Risk,Customers with low credit scores,2023-01-01T00:00:00Z
S008,C1001,Loyal,Consistent activity for over 5 years,2023-01-01T00:00:00Z
S009,C1002,Credit_Risk,Customers with low credit scores,2023-01-01T00:00:00Z
S0010,C1002,Credit_Risk,Customers with low credit scores,2023-01-01T00:00:00Z


In [0]:
customer_segmentation_df.write.format("delta").mode("overwrite").saveAsTable("hive_metastore.gold.Customer_segments")

## Aggrigations on Table

### 1.total and unique Count of segment name.

In [0]:
from pyspark.sql import functions as F
segment_distribution = customer_segmentation_df.groupBy("segment_name").agg(
    F.count("customer_id").alias("total_count"),
    F.countDistinct("customer_id").alias("unique_count")
)
segment_distribution.display()

segment_name,total_count,unique_count
High_Value,13,13
Loyal,581,175
Credit_Risk,933,288
New_User,26,10


###2. Get the sum,avg,min,max,count based on customer_id

In [0]:
from pyspark.sql import functions as F
customer_aggregations_df = transaction_df.groupBy("customer_id").agg(
    F.round(F.sum("amount"), 2).alias("total_amount"),
    F.round(F.avg("amount"), 2).alias("average_amount"),
    F.min("amount").alias("min_amount"),
    F.max("amount").alias("max_amount"),
    F.count("amount").alias("transaction_count")
)
customer_aggregations_df.display()

customer_id,total_amount,average_amount,min_amount,max_amount,transaction_count
C1804,234.04,39.01,2.78,71.18,6
C1875,134.89,33.72,4.03,61.31,4
C1602,165.59,55.2,18.44,75.79,3
C1524,109.49,36.5,21.09,57.33,3
C1100,114.31,38.1,6.63,70.99,3
C1842,93.53,31.18,12.45,51.83,3
C1628,40.83,20.41,19.38,21.45,2
C1829,170.52,34.1,12.47,63.94,5
C1305,26.48,26.48,26.48,26.48,1
C1774,92.38,46.19,19.95,72.43,2


### 3.agg on Currency from transaction_table

In [0]:
from pyspark.sql import functions as F

def analyze_currency_data(transactions_df):
    currency_analysis = transactions_df.groupBy("currency") \
        .agg(
            F.sum("amount").alias("total_amount_by_currency"),
            F.avg("amount").alias("avg_amount_by_currency")
        )

    return currency_analysis

currency_analysis_df = analyze_currency_data(transaction_df)
currency_analysis_df.display()


currency,total_amount_by_currency,avg_amount_by_currency
USD,8395723.594220996,2798.5745314069986


In [0]:
from pyspark.sql import functions as F
transaction_type_aggregations_df = transaction_df.groupBy("transaction_type").agg(
    F.round(F.sum("amount"), 2).alias("total_amount"),
    F.round(F.avg("amount"), 2).alias("average_amount"),
    F.min("amount").alias("min_amount"),
    F.max("amount").alias("max_amount"),
    F.count("amount").alias("transaction_count")
)
transaction_type_aggregations_df.display()


transaction_type,total_amount,average_amount,min_amount,max_amount,transaction_count
TRANSFER,2359988.34,3084.95,1.0,107801.54,765
DEPOSIT,2369029.42,3214.42,1.0,124454.6,737
WITHDRAWAL,2226424.28,2929.51,1.0,128300.66,760
PAYMENT,1440281.56,1951.6,1.0,94278.03,738


In [0]:
from pyspark.sql import functions as F
transaction_df = transaction_df.withColumn("year", F.year("timestamp"))
transaction_df = transaction_df.withColumn("month", F.month("timestamp"))
year_month_aggregations_df = transaction_df.groupBy("year", "month").agg(
    F.round(F.sum("amount"), 2).alias("total_amount"),
    F.round(F.avg("amount"), 2).alias("average_amount"),
    F.min("amount").alias("min_amount"),
    F.max("amount").alias("max_amount"),
    F.count("amount").alias("transaction_count")
)
year_month_aggregations_df.display()


year,month,total_amount,average_amount,min_amount,max_amount,transaction_count
2023,2,3802676.77,2763.57,1.0,128300.66,1376
2023,1,4235137.31,2782.61,1.0,124454.6,1522
2023,3,357909.52,3508.92,1.1,97065.97,102


## Agg on join_date from customer_Table

In [0]:
from pyspark.sql import functions as F
customer_df = customer_df.withColumn("join_year", F.year("join_date"))
joined_df = transaction_df.join(customer_df, on="customer_id", how="inner")
year_month_aggregations_df = joined_df.groupBy("join_year").agg(
    F.round(F.sum("amount"), 2).alias("total_amount"),
    F.round(F.avg("amount"), 2).alias("average_amount"),
    F.min("amount").alias("min_amount"),
    F.max("amount").alias("max_amount"),
    F.count("amount").alias("transaction_count")
)
year_month_aggregations_df.display()


join_year,total_amount,average_amount,min_amount,max_amount,transaction_count
2018,1017817.78,1953.58,1.0,128300.66,521
2022,639972.4,1457.8,1.0,98557.41,439
2019,1687596.52,4056.72,1.0,126743.93,416
2020,1654549.87,2918.08,1.0,124454.6,567
2017,1841092.26,3168.83,1.0,97065.97,581
2021,1554694.77,3266.17,1.0,125253.77,476


In [0]:
from pyspark.sql import functions as F
customer_segments_df = transaction_df.groupBy("customer_id").agg(
    F.round(F.sum("amount"), 2).alias("total_amount")
).withColumn(
    "segment",
    F.expr("CASE WHEN total_amount <= 1000 THEN 'Low Value' "
           "WHEN total_amount <= 5000 THEN 'Medium Value' "
           "ELSE 'High Value' END")
)
customer_segments_df.display()


customer_id,total_amount,segment
C1804,234.04,Low Value
C1875,134.89,Low Value
C1602,165.59,Low Value
C1524,109.49,Low Value
C1100,114.31,Low Value
C1842,93.53,Low Value
C1628,40.83,Low Value
C1829,170.52,Low Value
C1305,26.48,Low Value
C1774,92.38,Low Value


####Daily transaction volume

In [0]:
from pyspark.sql import functions as F


daily_volume = transaction_df_cleaned.groupBy(F.date_trunc('day', 'timestamp').alias('day')).agg(
    F.sum('amount').alias('total_volume'),
    F.count('transaction_id').alias('transaction_count')
)
daily_volume.display()

day,total_volume,transaction_count
2023-01-05T00:00:00Z,144503.83695566654,45
2023-01-20T00:00:00Z,13928.789614200592,52
2023-01-08T00:00:00Z,179715.09124934673,49
2023-01-17T00:00:00Z,127313.40093874931,44
2023-01-26T00:00:00Z,32449.480460882187,48
2023-01-06T00:00:00Z,259853.8012394905,48
2023-01-23T00:00:00Z,300973.4696198702,47
2023-01-01T00:00:00Z,1410.8499903678894,47
2023-01-13T00:00:00Z,78897.4529747963,59
2023-01-22T00:00:00Z,135503.9815375805,50


### weekly transaction volume

In [0]:
weekly_volume = transaction_df_cleaned.groupBy(F.date_trunc('week', 'timestamp').alias('week')).agg(
    F.sum('amount').alias('total_volume'),
    F.count('transaction_id').alias('transaction_count')
)
weekly_volume.display()

week,total_volume,transaction_count
2023-01-23T00:00:00Z,1001274.5882818699,336
2023-01-02T00:00:00Z,994626.7481119633,338
2023-01-09T00:00:00Z,941291.9074559212,355
2023-01-30T00:00:00Z,1106369.9766197205,339
2022-12-26T00:00:00Z,1410.8499903678894,47
2023-01-16T00:00:00Z,864536.8983428478,344
2023-02-06T00:00:00Z,974755.3784345388,339
2023-02-13T00:00:00Z,624410.7418041229,363
2023-02-20T00:00:00Z,1256468.9532836676,337
2023-02-27T00:00:00Z,630577.5518959761,202


#### Monthly transaction volume

In [0]:

monthly_volume = transaction_df_cleaned.groupBy(F.date_trunc('month', 'timestamp').alias('month')).agg(
    F.sum('amount').alias('total_volume'),
    F.count('transaction_id').alias('transaction_count')
)
monthly_volume.display()

month,total_volume,transaction_count
2023-01-01T00:00:00Z,4235137.305144787,1522
2023-02-01T00:00:00Z,3802676.7678308487,1376
2023-03-01T00:00:00Z,357909.5212453604,102
